In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import csv

from tqdm import tqdm

2024-08-04 12:45:04.047681: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-04 12:45:04.077344: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 12:45:04.077392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 12:45:04.078362: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-04 12:45:04.084137: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

from sklearn.preprocessing import OneHotEncoder

In [4]:
# GPU settings



gpus = tf.config.list_physical_devices('GPU')



# Setting gpu[0] as main device
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)


# Setting memory growth
try:
  tf.config.experimental.set_memory_growth(gpus[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass



# tf.debugging.set_log_device_placement(True)


os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

1 Physical GPUs, 1 Logical GPU
cuda_malloc_async


2024-08-04 12:45:06.005073: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-04 12:45:06.038537: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-04 12:45:06.038712: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
train_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/train_synsets"
valid_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/valid_synsets"
test_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/test_synsets"

model_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Models"

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (Image_height, Image_width),
    color_mode="rgb",
    batch_size=batch_size,
    seed=1,
    shuffle=True,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(Image_height, Image_width),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(Image_height, Image_width),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

train_num = train_generator.samples
valid_num = valid_generator.samples
train_num = train_generator.samples

NameError: name 'Image_height' is not defined

In [ ]:
printf()

In [ ]:
class VGGNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()
        
        self.add(Conv2D(
            64,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu",
            input_shape=input_shape
        ))

        self.add(Conv2D(
            64,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Flatten()),

        self.add(Dense(4096, activation="relu"))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(num_classes, activation="softmax"))

        self.compile(
            optimizer=Adam(0.001),
            loss = "categorical_crossentropy",
            metrics=["accuracy"]
        )

In [ ]:
num_epochs = 5
batch_size = 1
Image_height = 224
Image_width = 224
Input_shape = (Image_width, Image_height, 3)
num_classes = 3


model = VGGNet(Input_shape, num_classes=num_classes)

model.summary()

In [ ]:
log_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%M%Y-%H%M%S")
tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks_list = [tensorboard_callbacks]

In [ ]:
# Since lack of gpu power, we cant train this model in local machine, so we take step to take pretrained model

model.fit(
    train_generator,
    epochs=num_epochs,
    # steps_per_epoch=train_images_num//batch_size,
    validation_data=valid_generator,
    # validation_steps=valid_images_num//batch_size,
    callbacks=callbacks_list,
    verbose=1
)

In [ ]:
# from keras.applications.vgg16 import VGG16
# vggmodel = VGG16(weights='imagenet', include_top=True)

In [ ]:
# print("layers of the model:")
# for la in vggmodel.layers:
#     print(la)


# print("\n\nlayers that we are going to set trainable to False")
# for layers in (vggmodel.layers)[:15]:
#     print(layers)
#     layers.trainable = False

# X= vggmodel.layers[-2].output
# predictions = Dense(2, activation="softmax")(X)
# model_final = Model(vggmodel.input, predictions)
# opt = Adam(learning_rate=0.0001)
# model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
# model_final.summary()